In [22]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import warnings

### 定义功能函数

In [23]:
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

def is_interactive_notebook():
    return __name__ == "__main__"
def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)
def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)
class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None
    def step(self):
        None
    def zero_grad(self, set_to_none=False):
        None

class DummyScheduler:
    def step(self):
        None

### 定义深拷贝

In [24]:
def clones(module, N):
    '''
    Produce N identical layers.
        copy.deepcopy() 是 Python 的 copy 模块中的一个函数，它用于创建一个对象的深拷贝。深拷贝意味着：
            这个新拷贝的对象与原对象完全独立，它是原对象的一个递归副本，所有内部对象也会被拷贝。
            修改新对象不会影响原对象，反之亦然。
        nn.ModuleList 是 PyTorch 提供的一个特殊的列表，用于存放多个 nn.Module 实例。
            与 Python 的普通 list 不同, ModuleList 被设计为能与 PyTorch 的其他模块(如优化器、损失函数等)很好地配合使用。
            它的作用类似于一个容器，可以将多个子模块组织在一起并注册到 nn.Module 系统中。
    '''
    
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


### Encoder-Decoder 架构

<img src="./images/transformer.png" alt="示例图片" width="600">

In [25]:
class EncoderDecoder(nn.Module):
    '''
    A standard Encoder-Decoder architecture. Base for this and many other models.
    '''
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__() # 其作用是让当前类(EncoderDecoder)调用其父类的构造函数(__init__)，从而确保父类中的初始化逻辑得以执行。
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        '''
        Take in and process masked src and target sequences.
        '''
        
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    
    '''
    class Encoder(self, layer, N):
        forward(self, x, mask):
        
    class EncoderLayer(self, size, self_attn, feed_forward, dropout):
        forward(self, x, mask):
    '''
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    
    '''
    class Decoder(self, layer, N):
        forward(self, x, memory, src_mask, tgt_mask):
    
    class DecoderLayer(self, size, self_attn, src_attn, feed_forward, dropout):
        forward(self, x, memory, src_mask, tgt_mask):
    '''
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

### 生成器 Generator

<img src="./images/Generator.png" alt="示例图片" width="600">

In [26]:
class Generator(nn.Module):
    '''
    Define standard linear + softmax generator step
    '''
    
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
        
    def forward(self, x):
        return log_softmax(self.proj(x), dim = -1)

### LayerNormalization

$LN(x_i) = \alpha * \frac{x_i  - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}} + \beta , \quad\quad\quad BN(x_i) = \alpha * \frac{x_i  - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} + \beta$


<img src="./images/LayerNorm.png" alt="示例图片" width="600">

In [56]:
class LayerNorm(nn.Module):
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        # nn.Parameter()：这是 PyTorch 中的一种特殊张量，它被自动注册为模型的可学习参数。使用这种张量，PyTorch 的优化器会自动更新它的值。
        # torch.ones(features)：生成一个大小为 features 的全为 1 的张量。a_2 作为缩放系数，将用于对归一化后的输出进行缩放。
        self.a_2 = nn.Parameter(torch.ones(features))
        # b_2 作为偏置项，将用于对归一化后的输出进行平移。
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
        
    def forward(self, x):
        # x：这是输入张量。通常，它的形状为 [batch_size, ..., features]，其中 features 是最后一个维度的大小
        # keepdim=True：确保均值计算后保留原始张量的维度，使得输出的形状与输入一致。这样在后续计算中，张量的广播机制可以正常工作。
        mean = x.mean(-1, keepdim=True)
        # 这一行计算输入 x 的标准差（std），用法与计算均值类似。
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

### 定义子层框架 norm -> (any) sublayer -> dropout -> residual

<img src="./images/Sublayer.png" alt="示例图片" width="200">

In [57]:
class SublayerConnection(nn.Module):
    '''
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last
    Each layer has two sub-layers. The first is a multi-head self-attention mechanism, 
    and the second is a simple, position-wise fully connected feed-forward network.
    '''
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        # nn.Dropout(dropout) 是 PyTorch 中的 Dropout 层，用于在训练过程中对神经网络的部分神经元进行随机失活（dropout）
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        '''
        Note for code simplicity the norm is first as opposed to last
        注意，为了代码的简单性，规范是第一位的，而不是最后一位
        norm -> (any) sublayer -> dropout -> residual
        '''
        return x + self.dropout(sublayer(self.norm))

### Attention

<img src="./images/attention.png" alt="示例图片" width="700">

$$
Attention(Q, K, V) = softmax(\frac{QK^\top}{\sqrt{d_k}})V
$$

In [58]:
def attention(query, key, value, mask=None, dropout=None):
    '''
    Compute 'Scaled Dot Product Attention'.
    The input consists of queries and keys of dimension d_k, and values of dimension d_v.
    K = W_k * x
    x: 这是输入张量。通常, 它的形状为 [batch_size, ..., features], 其中 features 是最后一个维度的大小
    所以Q, K也是[batch_size, ... , features]
    '''
    
    # 这通常是特征的维度，代表键和值的维度（d_k）。这个值用于缩放得分，以防止注意力得分过大。
    d_k = query.size(-1)
    # torch.matmul() 计算点积得分。它将两个相同维度的向量进行运算，产生一个标量（单一数字）.
    # key.transpose(-2, -1) 将 key 的最后两个维度调换，使得它的形状变为 [batch_size, ..., d_k]，与 query 的形状匹配。
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # 使用 masked_fill 函数将 scores 中对应于 mask 中为 0 的位置填充为 -1e9，以确保在 softmax 计算时这些位置的注意力分数变为 0。这样做的目的是防止模型关注这些被屏蔽的输入.
        scores = scores.masked_fill(mask == 0, -1e9)
    # 计算注意力权重（即对每个查询的注意力分配）。dim=-1 表示在最后一个维度上进行 softmax 操作。
    alpha_attn = scores.softmax(dim=-1)
    if dropout is not None:
        alpha_attn = dropout(alpha_attn)
    return torch.matmul(alpha_attn, value), alpha_attn

### Multi Head Attention

<img src="./images/MultiHeadAttention.png" alt="示例图片" width="700">

$MultiHead(Q, K, V) = Concat(head_1, head_2, ... , head_h)W^O\text{, where }head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)$

$W_i^Q \in \mathbb{R}^{d_{model} \times d_k}, \quad W_i^K \in \mathbb{R}^{d_{model} \times d_K}, \quad W_i^V \in \mathbb{R}^{d_{model} \times d_V}, \quad W_i^O \in \mathbb{R}^{hd_v \times d_{model}}$

In [59]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        
        # 如果 d_model 为 512，则输入向量的每个元素都由 512 维的特征组成。
        # 在多头自注意力机制中，d_model 会被划分成多个头（如 h 个头），每个头处理(d_model / h)维的输入。因此, d_model 需要能够被头数整除，以便均匀划分。
        assert d_model % h == 0
        # We assume d_v always equals d_k
        # //: 这是整除运算符，确保结果为整数。也就是说，d_model 会被头数 h 整除，计算出每个头的特征维度。
        self.d_k = d_model // h 
        self.h = h
        # linears: W_i^Q, W_i^K, W_i^V, W_i^O
        self.linears = clones(nn.Linear(d_model, d_model), 4) 
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        '''
        Implements Figure
        '''
        if mask is not None:
            # Some mask applied to all h heads
            # unsqueeze 是一个非常有用的张量操作，它用于在指定位置添加一个新的维度。
            # mask.unsqueeze(1)：如果提供了 mask，则通过 unsqueeze 在第 1 维度添加一个维度，目的是为多个头共享相同的 mask。
            mask = mask.unsqueeze(1)
        # K = W_k * x
        # x: 这是输入张量。通常, 它的形状为 [batch_size, ..., features], 其中 features 是最后一个维度的大小
        # 所以Q, K也是[batch_size, ... , features]
        # 获取批量大小（nbatches），通常是 query 的第一个维度，表示当前 batch 中有多少个样本。
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k
        '''
        这段代码的作用是通过线性变换，将 query、key 和 value 映射到多头注意力机制的特征空间。
        view(nbatches, -1, self.h, self.d_k):
            .view(nbatches, -1, self.h, self.d_k)：将线性变换的结果重塑为 (nbatches, seq_len, num_heads, head_dim) 的形状，目的是分配给每个头。
            view 函数用于改变张量的形状。这里的目的是将每个线性层的输出重塑为形状为 [nbatches, -1, self.h, self.d_k] 的张量。
            nbatches 是批次大小，表示每个批次中的样本数。
            -1 是自动推导的维度，它会根据原始张量的总元素数量和指定的其他维度自动计算,即 seq_len。
            self.h 是注意力头的数量。
            self.d_k 是每个头的特征维度。
        .transpose(1, 2)：交换维度，方便后续对多个注意力头并行处理。 -> (nbatches, num_heads, seq_len, head_dim) 
        zip 函数将 self.linears 和 (query, key, value) 这三个张量打包在一起，使得在循环中可以同时迭代线性层和输入张量。
            三个线性层, 分别以query, key, value作为参数跑一遍
        '''
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for lin, x in zip(self.linears, (query, key, value))
        ]
        
        # 2) Apply attention on all the projected vectors in batch.
        '''
        Our pre-defined function attention:
            attention(query, key, value, mask=None, dropout=None):
            return torch.matmul(alpha_attn, value), alpha_attn
        '''
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear
        '''
        x.transpose(1, 2)：将维度换回来，将注意力头的输出重新排列，使得每个样本的所有头的输出排列在一起。
        .contiguous()：保证张量在内存中是连续的，以便后续的 .view() 操作。 (nbatches, num_heads, seq_len, head_dim)  -> (nbatches, seq_len, num_heads, head_dim)
        .view(nbatches, -1, self.h * self.d_k)：将张量重塑为形状 (nbatches, seq_len, d_model)，将所有头的输出拼接到一起。
        '''
        x = (
            x.transpose(1, 2)
            .contiguoous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        
        '''
        del 是一个用于删除对象的关键字。它可以删除变量、列表中的元素, 甚至删除对象的属性。
        del 并不会直接释放内存，而是通过解除对象的引用，使得 Python 的垃圾回收机制可以在适当的时候回收那些不再被引用的对象。
        '''
        del query
        del key
        del value
        
        return self.linears[-1](x) # W_i^O
        

# Applications of Attention in our Model

1. 在“Encoder-Decoder Attention” Layer 中，查询来自于前一个 Decoder Layer，而内存的 Key 和 Value 则来自 Encoder 的输出。这使得 Decoder 中的每个位置都能够关注输入序列中的所有位置。这模仿了 Seq2Seq 模型中典型的 Encoder-Decoder 注意力机制。

2. Encoder 包含 Self-Attention Layer 。在 Self-Attention Layer 中，所有的 Key、Value 和 Query 都来自同一个地方，在这种情况下，是 Encoder 中前一层的输出。Decoder 中的每个位置都可以关注编码器前一层中的所有位置。

3. 同样，Decoder 中的 Self-Attention Layer 允许 Decoder 中的每个位置关注到该位置及之前的所有位置。我们需要防止 Decoder 中的左向信息流动，以保持 auto-regressive 特性。我们通过在缩放的点积注意力 (scaled dot-product attention) 中对输入 softmax 的所有非法连接对应的值进行 mask (设置为 -∞) 来实现这一点。

### Position-wise Feed-Forward Networks

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad FFN(x) = ReLU(xW_1 + b_1)W2 + b_2 = max(0, xW_1 + b_1)W_2 + b_2$

The dimensionality of input and output is $d_{model} = 512$, and the inner-layer has dimensionality $d_{ff}$

<img src="./images/FFN.png" alt="示例图片" width="250">

In [60]:
class PositionwiseFeedForward(nn.Module):
    '''
    Implement FFN equation.
    '''
    
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # return self.w_2(self.dropout(self.w_1(x).relu()))
        return self.w_2(self.dropout(max(0, self.w_1(x))))

### Embeddings and Softmax

<img src="./images/Embedding.png" alt="示例图片" width="200">

<img src="./images/Embedding2.png" alt="示例图片" width="172">

In [61]:
class Embeddings(nn.Module):
    '''
    与其他序列转换模型类似, 我们使用学习的 embeddings 将输入 tokens 和输出 tokens 转换为维度向量。我们还使用常用的学习线性变换和 softmax 函数将 Decoder 的输出转换为预测的下一个令牌概率。
    在我们的模型中, 我们在两个 Embedding Layers 和 pre-softmax 线性变换之间共享相同的权重矩阵
    '''
    
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        '''
        lut 通常是 Look-Up Table(查找表）)的缩写, 用于将离散的词索引映射到连续的高维向量空间。
        在机器学习和深度学习中, 查找表是用于存储和检索数据的一种数据结构。它通过一个键(key)来获取对应的值(value)。
        对于词嵌入层，词索引(即每个词在词汇表中的位置)充当键，而嵌入向量则是与这些索引关联的值。
        
        d_model: 嵌入向量的维度。
        vocab: 词汇表的大小，即可以表示的不同词的数量。
        '''
        self.lut = nn.Embedding(d_model, vocab)
        self.d_model = d_model
    
    def forward(self, x):
        # 在 Embedding Layers 中，我们将这些权重乘以 根号(d_model)
        # 这一步是为了对嵌入进行缩放，以增加数值稳定性。通常，乘以 sqrt(d_model) 有助于在后续的注意力机制中保持均匀的梯度分布，尤其是在使用 softmax 时，避免由于输入值过小而导致的梯度消失问题。
        return self.lut(x) * math.sqrt(self.d_model)
    

### Positional Encoding

<img src="./images/Position.png" alt="示例图片" width="700">

<img src="./images/Position2.png" alt="示例图片" width="300">

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad PE_{(pos, 2i)} = \sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad PE_{(pos, 2i+1)} = \cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad$ 其中 2i 和 2i+1 是特征的维度第几维，分为第**奇**数维和第**偶**数维

<img src="./images/Position3.png" alt="示例图片" width="700">

In [62]:
class PositionalEncoding(nn.Module):
    '''
    Implement the PE function.
    d_model: 输入序列中每个词嵌入的维度（与词向量的维度一致）
    '''
    
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        
        # compute the positional encoding once in log space.
        # 初始化一个大小为 [max_len, d_model] 的全零张量 pe，用于存储位置编码的值。
        pe = torch.zeros(max_len, d_model)
        # torch.arange 是 PyTorch 中用于生成等间隔数值序列的一种函数。其功能类似于 Python 中的 range 函数，但它生成的是张量（tensor）而不是普通的数字序列。
        # 生成一个 position 张量，它的值是从 0 到 max_len-1 的序列（即每个位置的索引），形状为 [max_len, 1]。unsqueeze(1) 表示在第一维度增加一个维度。
        position = torch.arange(0, max_len).unsqueeze(1) # [max_len, 1]
        '''
        计算位置编码中的分母项 div_term, 使用上述公式, 保存为 div_term = 1/分母。
        torch.arange(0, d_model, 2) 生成从 0 开始到 d_model-1 步长为 2 的序列，这部分用来处理偶数位置的维度。
        
        为什么是公式: torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model) ?

        '''
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term) # (0, 2, 4, ...)
        pe[:, 1::2] = torch.cos(position * div_term) # (1, 3, 5, ...)
        
        # 扩展维度并注册为 buffer
        # 使用 unsqueeze(0) 为位置编码增加一个批次维度, 这样可以适配不同批次的数据。
        pe = pe.unsqueeze(0) # [max_len, d_model] -> [1, max_len, d_model]
        
        '''
        将 pe 作为 buffer 注册到模型中。Buffer 是模型的持久状态，但不会作为模型参数参与优化，也不会随着梯度更新而改变。
        通常情况下，模型中的状态分为两类：
            可训练参数：这些参数会在模型的前向传播和反向传播过程中参与梯度计算，并在优化器中更新。这些参数可以通过 nn.Parameter 来注册，它们通常是权重或偏置。通过优化器或手动调整更新
            非可训练的持久状态：这些是模型需要的值，但它们不需要参与优化过程。只能通过代码逻辑手动修改。例如： 
                用于存储固定的常量，比如用于正则化的参数、批次统计数据（如 BatchNorm 中的均值和方差）。
                用于模型中不需要梯度更新但要随模型一起保存的变量。
        Buffer 就属于第二类状态。它的关键点是：
            不会参与梯度计算，不会随着训练过程中的优化步骤被更新。
            会随模型保存和加载，即使这些变量不会更新，仍然希望它们作为模型的一部分存储和恢复。
        持久存储: pe 是一个位置编码矩阵，代表了每个位置的编码。这个矩阵是预先计算的，并且在整个模型中保持不变，因此它不需要被优化器更新。
                但在保存模型时，我们希望位置编码 pe 和其他可训练的参数一样，被保存下来，并在加载模型时恢复。
        不会更新：由于位置编码的矩阵 pe 是基于固定的公式计算的，它不需要参与反向传播和梯度更新。
                因此，将其作为 buffer 来注册，这样在训练过程中它不会被优化器错误地修改。
        '''
         
        self.register_buffer("pe", pe)
        
    def forward(self, x):
        # x: [batch_size, sequence_length, d_model]
        # 这一步是将位置编码加到词嵌入上，使得模型能够感知位置信息。
        # requires_grad_(False) 是为了防止位置编码参与梯度更新，因为它们是固定的。
        x =  x + self.pe[:, x.size(1)].requires_grad_(False)
        
        '''
        为什么对位置编码应用 Dropout ? 
            尽管 Dropout 最初是在全连接层中用于“随机失活”一部分神经元的，但它的应用范围实际上更广，既可以用于神经网络的各层、或输出层，也可以用于特征向量，如输入的词嵌入或位置编码。
            增加随机性: 通过对位置编码的部分值进行随机“失活”，可以引入一些随机性，使得模型不会过度依赖特定的位置信息。这可以让模型在训练时更具鲁棒性，减少过拟合的可能。
            正则化输入: Dropout 可以防止模型对特定的输入位置编码产生过度拟合。因为位置编码是添加到输入中的一部分，
                      对其进行 dropout 可以有效地打破模型对绝对位置信息的依赖性，迫使模型更关注整体上下文，而不仅仅是某些特定位置的依赖。
        '''
        return self.dropout(x)

### 展示 Positional Encoding

<img src="./images/Position_example.png" alt="示例图片" width="700">

In [63]:
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))
    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )
    
    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )

# show_example(example_positional())

### Encoder 框架

The encoder is composed of a stack of N = 6 identical layers.

<img src="./images/Encoder.png" alt="示例图片" width="250">

In [64]:
class Encoder(nn.Module):
    '''
    Core encoder is a stack of N layers
    layer = class EncoderLayer(self, size, self_attn, feed_forward, dropout):
                forward(self, x, mask):
    '''
    
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        '''
        Pass the input (and mask) through each layer in turn
        '''
        
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

### Encoder 中的每一层

<img src="./images/Encoder.png" alt="示例图片" width="250">

In [65]:
class EncoderLayer(nn.Module):
    '''
    Encoder is made up of self-attn and feed forward (defined below).
    这个是 class Encoder 中的layer
    '''
    
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self. dropout = dropout
        '''
        class SublayerConnection(self, size, dropout):
            forward(self, x, sublayer):
                return x + self.dropout(sublayer(self.norm))
        '''
        self.sublayer = clones(SublayerConnection(size, dropout), 2) # self-attention and feed forward
        self.size = size
        
    def forward(self, x, mask):
        '''
        Follow Figure for connections.
        
        sublayer[0] 代表一个可调用的子层, 并且该子层接受两个参数: x 和通过 Lambda 表达式计算的另一个函数调用的结果。
        这一部分是一个 Lambda 表达式，它创建了一个匿名函数，并把这个匿名函数作为参数传递给 sublayer[0]
        self.self_attn(x, x, x, mask)：调用当前类的 self_attn 函数, 它是自注意力机制(self-attention)的实现。
            第一个 x: 是查询 Q(query)。
            第二个 x: 是键 K(key)。
            第三个 x: 是值 V(value)。
            mask: 通常是用于屏蔽某些不需要关注的部分, 比如在解码器中的解码屏蔽机制, 或用于处理填充的部分。
        '''
        
        x = self.sublayer[0](x, lambda x : self.self_attn(x, x, x, mask)) # 在 self_attn 内部实现了 W_Q, W_K, W_V 的相乘和 Q, K, V 的计算，所以这里只需传 x, x, x.
        return self.sublayer[1](x, self.feed_forward)

### Decoder 框架

The decoder is also composed of a stack of N = 6 identical layers.

<img src="./images/Decoder.png" alt="示例图片" width="250">

<img src="./images/Decoder2.png" alt="示例图片" width="900">

In [66]:
class Decoder(nn.Module):
    '''
    Generic N layer decoder with masking
    class LayerNorm(self, features, eps=1e-6)
        forward(self, x):
    layer = class DecoderLayer(self, size, self_attn, src_attn, feed_forward, dropout):
                forward(self, x, memory, src_mask, tgt_mask):
    '''
    
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = self.layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

### Decoder 中的每一层

<img src="./images/Decoder.png" alt="示例图片" width="250">

<img src="./images/Decoder2.png" alt="示例图片" width="900">

In [71]:
class DecoderLayer(nn.Module):
    '''
    Decoder is made of self-attn, src-attn and feed forward (defined below)
    除了每个Encoder层中的两个子层之外, 解码器还插入第三子层, 该第三子层对Encoder栈的输出执行多头注意。类似于Encoder, 我们在每个子层周围使用残差连接, 然后进行层归一化。
    
    这个是 class Decoder 中的layer
    '''
    
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        '''
        class SublayerConnection(self, size, dropout):
            forward(self, x, sublayer):
                return x + self.dropout(sublayer(self.norm))
        '''
        self.sublayer = clones(SublayerConnection(size, dropout), 3) # self-attention, src-attention and feed forward
        
    def forward(self, x, memory, src_mask, tgt_mask):
        '''
        Follow Figure for connections.
        
        self_attn 是 Decoder 内部的自注意力, 用于当前 Decoder 状态的自我计算。
        src_attn 是 Decoder 对 Encoder 输出的注意力, 帮助 Decoder 利用编码器的上下文信息。
        src_mask 用于过滤掉 Encoder 的输出 memory 的填充位置。
        tgt_mask 用于对目标序列进行掩码，防止模型关注无效的填充部分，以及阻止 Decoder 在训练时看到未来的词。
        '''
        m = memory
        # self_attn 和 src_attn 实现是一样的, 只是传入的参数不同
        x = self.sublayer[0](x, lambda x : self.self_attn(x, x, x, tgt_mask)) # 在 self_attn 内部实现了 W_Q, W_K, W_V 的相乘和 Q, K, V 的计算，所以这里只需传 x, x, x.
        x = self.sublayer[1](x, lambda x : self.src_attn(x, m, m, src_mask)) # 在 self_attn 内部实现了 W_Q, W_K, W_V 的相乘和 Q, K, V 的计算，所以这里只需传 x, m, m.
        return self.sublayer[2](x, self.feed_forward)
        
        

### 展示 Mask

<img src="./images/mask.png" alt="示例图片" width="500">

In [72]:
def subsequent_mask(size):
    '''
    Mask out subsequent positions.
    这个函数用于生成一个 "上三角掩码"，防止解码器在自注意力机制中关注未来的时间步。掩码矩阵的对角线及其以下部分为 True(可以关注), 对角线上方的部分为 False(不能关注)。
    '''
    
    # 1：第一个维度通常是批次大小，这里设为 1 是因为该掩码在所有批次上共享。 size：第二个和第三个维度表示要处理的序列长度。
    attn_shape = (1, size, size)
    '''
    torch.ones(attn_shape)：首先生成一个形状为 (1, size, size) 的全 1 张量。这是一个全 1 的矩阵，表示初始的注意力权重矩阵。
    torch.triu(..., diagonal=1): 
        torch.triu 是一个函数，用于获取上三角矩阵。它会将张量中所有对角线以下的元素置为 0。
        参数 diagonal=1 表示生成一个上三角矩阵，但对角线及其以下的部分为 0。对角线上的元素和其下方的元素被遮蔽, 只有对角线右上方的元素是保留的 1。
        这样能保证在注意力机制中，每个时间步只能关注自己和之前的时间步，而不能关注未来的时间步
    .type(torch.uint8)：将张量的类型转换为 torch.uint8。这个数据类型通常用于表示掩码, 因为掩码通常是布尔值或 0/1 值。
    '''
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    
    '''
    这里返回一个布尔张量，它将上一步中的 subsequent_mask 与 0 进行比较，生成一个布尔掩码。
    对于那些 subsequent_mask 中值为 0 的位置，比较操作 subsequent_mask == 0 将返回 True, 这些位置将不会被掩盖。
    对于值为 1 的位置，比较操作返回 False, 这些位置会被掩盖。
    最终返回的张量是一个大小为 (1, size, size) 的布尔掩码，表示哪些位置可以被注意力关注，哪些位置需要被遮蔽。
    '''
    return subsequent_mask == 0

def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(), # (1, size, size)
                    "Window": y,
                    "Masking": x,
                }
                for y in range(20)
                for x in range(20)
            )
        ]
    )
    
    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )
    
# show_example(example_mask())

# Full Model

<img src="./images/transformer.png" alt="示例图片" width="600">

### Xavier 初始化

它通过均匀分布对权重矩阵进行初始化，权重的取值范围为[−a,a]，假设 in_features 和 out_features 分别是输入和输出的维度。其中：

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad a = \sqrt{\frac{6}{in\_features \quad+\quad out\_features}}$

In [73]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2024, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    '''
    class MultiHeadAttention(self, h, d_model, dropout=0.1):
        forward(self, query, key, value, mask=None):
    '''
    attn = MultiHeadAttention(h, d_model)
    '''
    class PositionwiseFeedForward(self, d_model, d_ff, dropout=0.1):
        forward(self, x):
    '''
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    '''
    class PositionalEncoding(self, d_model, dropout, max_len=5000):
        forward(self, x):
    '''
    position = PositionalEncoding(d_model, dropout)
    '''
    class EncoderDecoder(self, encoder, decoder, src_embed, tgt_embed, generator):
        forward(self, src, tgt, src_mask, tgt_mask):
        
        def encode(self, src, src_mask):
            return self.encoder(self.src_embed(src), src_mask)
            
        def decode(self, memory, src_mask, tgt, tgt_mask):
            return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
            
    class Encoder(self, layer, N):
        forward(self, x, mask):
    class EncoderLayer(self, size, self_attn, feed_forward, dropout):
        forward(self, x, mask):
            
    class Decoder(self, layer, N):
        forward(self, x, memory, src_mask, tgt_mask):
    class DecoderLayer(self, size, self_attn, src_attn, feed_forward, dropout):
        forward(self, x, memory, src_mask, tgt_mask):
    '''
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N), # self_attn 和 src_attn 实现是一样的, 只是传入的参数不同
        # nn.Sequential() 可以将多个 nn.Module 对象按顺序组合为一个新的模块。
        # 当输入通过该组合模块时，会按照定义的顺序依次通过每个子模块。
        # 这对构建简单的神经网络结构非常有用，特别是那些具有线性层堆叠的模型。
        # class Embeddings(self, d_model, vocab):
        #     forward(self, x):
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings((d_model, tgt_vocab), c(position))),
        # class Generator(self, d_model, vocab):
        #     forward(self, x):
        Generator(d_model, tgt_vocab),
    )
    
    '''
    它对模型中所有维度大于1的参数(通常是权重矩阵)使用 Xavier 初始化(或称为 Glorot 初始化)，以保证网络的训练效果更好。
    model.parameters()：返回模型中所有可训练参数的迭代器。这些参数通常包括模型中的权重矩阵和偏置向量。
    '''
    for p in model.parameters():
        if p.dim() > 1: # 检查参数的维度，如果维度大于 1。p.dim()：返回参数张量的维度。
            '''
            nn.init.xavier_uniform_()：是 PyTorch 中的一种初始化方法，也称为 Xavier 初始化。
            详情见 Markdown 公式
            Xavier 初始化的作用: Xavier 初始化的目的是让每一层的输入和输出方差保持一致，防止信号在前向传播时快速衰减或爆炸。
            这种初始化在神经网络的早期层和深度网络的训练中非常有效，特别是在没有使用 ReLU 激活函数的情况下。
            nn.init.xavier_uniform_ 是一个 就地(in-place) 操作，意味着它直接修改传入的参数张量(例如，权重矩阵)。
                在这个函数名的末尾有一个下划线(_), 这表示这个函数会修改传入的对象, 而不是返回一个新的对象。
            nn.init.xavier_uniform 是一个 函数，用于返回一个 "新的张量"，其元素使用 Xavier 均匀分布进行初始化。
                这个函数没有下划线，意味着它不会修改传入的参数，而是返回一个新的张量。
            '''
            nn.init.xavier_uniform_(p)
    
    return model
    

## Inference

在这里，我们生成模型的预测。我们尝试使用我们的Transformer来记忆输入。由于模型尚未经过训练，因此输出是随机生成的。

In [78]:
def inference_test():
    
    # make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2024, h=8, dropout=0.1)
    test_model = make_model(11, 11, 2)
    # model.eval() 用于将模型设置为评估模式（evaluation mode）
    test_model.eval()
    # 创建一个包含输入序列的张量 src，表示源序列的 tokens，维度为 (1, 10)，表示 batch size 为 1，序列长度为 10。
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    # 创建一个全 1 的掩码 src_mask，用于表示源序列中所有位置都可以被注意到。
    src_mask = torch.ones(1, 1, 10)
    '''
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    '''
    memory = test_model.encode(src, src_mask)
    # ys 通常表示模型在推断（inference）阶段生成的输出
    # 初始化输出序列 ys，开始时只有一个 token（通常是一个开始符号），形状为 (1, 1)，数据类型与 src 相同。
    ys = torch.zeros(1, 1).type_as(src) # [batches, 输出答案长度]
    
    # range(9) 生成总共 10 个 tokens 的输出序列。
    for i in range(9):
        # 生成第i个输出序列
        '''
        def decode(self, memory, src_mask, tgt, tgt_mask):
            return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
        '''
        out = test_model.decode(
            # subsequent_mask(size) 生成一个 "上三角掩码"
            # 输入为 memory（编码器的输出），src_mask（源序列的掩码），ys（当前输出序列）
            # 以及通过 subsequent_mask(ys.size(1)).type_as(src.data) 生成的后续掩码。
            # 后续掩码用于确保解码器在生成输出时不会访问到未来的 tokens。
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        '''
        class Generator(self, d_model, vocab):
            forward(self, x):
        '''
        # 通过模型的生成器（通常是一个线性层加 softmax）将解码器输出的最后一个 token out[:, -1] 转换为概率分布 prob，
        # 表示每个可能的下一步 token 的概率。
        prob = test_model.generator(out[:, -1])
        #  从概率分布中找到概率最高的 token，得到其索引 next_word，这就是模型在当前步骤预测的下一个 token。
        _, next_word = torch.max(prob, dim=1)
        # .data 返回的是 next_word 张量的内容
        next_word = next_word.data[0]
        '''
        ys = torch.cat([...], dim=1): 将预测的 next_word 添加到 ys 中，扩展输出序列
        
        torch.empty(1, 1).type_as(src.data).fill(next_word):
            创建一个新张量来存放下一个 token, 然后与当前的 ys 进行拼接。
            torch.empty(size) 创建一个未初始化的张量，其形状为 size。在这里, size 是 (1, 1)，所以创建的是一个包含单个元素的二维张量。
            src.data 是一个张量, src 是在前面的代码段中定义的源输入
            .fill(value) 方法用于将张量中的所有元素填充为指定的值。在这里, next_word 是之前预测的下一个词的索引。
            所以，调用 fill(next_word) 会将这个未初始化的张量中的唯一元素设置为 next_word 的值。
            这整行代码的目的就是创建一个形状为 (1, 1) 的张量，初始化为 next_word 的值，并确保这个张量的数据类型和设备与 src 张量保持一致。
            最终的结果是一个包含单个元素(即预测的下一个词的索引)的张量, 用于在后续的步骤中与其他张量进行连接(concatenate)。
        '''
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill(next_word)], dim=1
        )
        
        print("Example Untrained Model Prediction: ", ys)
        
def run_tests():
    for _ in range(10):
        inference_test()
        
# show_example(run_tests)
    
        

<img src="./images/inference_test.png" alt="示例图片" width="600">